In [1]:
#Primero importamos librerías necesarias e instalamos Spark en el Notebook

import findspark
findspark.init()

import pandas as pd
import pyspark

In [2]:
#Importamos librerías de Pyspark y abrimos la sesión y contexto Spark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Df')\
        .getOrCreate()

In [3]:
#Importamos los datasets en .csv usando la función de Spark para csv
worldhapprep = spark.read.csv("C:/Users/Usuario/Desktop/big-data-processing/Proyecto-Final/Datasets/world-happiness-report.csv", sep=",", header=True, inferSchema=True)
worldhapprep21 = spark.read.csv("C:/Users/Usuario/Desktop/big-data-processing/Proyecto-Final/Datasets/world-happiness-report-2021.csv", sep=",", header=True, inferSchema=True)

In [4]:
#Chequeo visual
worldhapprep.show(10)

+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
|Country name|year|Life Ladder|Log GDP per capita|Social support|Healthy life expectancy at birth|Freedom to make life choices|Generosity|Perceptions of corruption|Positive affect|Negative affect|
+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
| Afghanistan|2008|      3.724|              7.37|         0.451|                            50.8|                       0.718|     0.168|                    0.882|          0.518|          0.258|
| Afghanistan|2009|      4.402|              7.54|         0.552|                            51.2|                       0.679|      0.19|                     0.85|          0.584|          0.237|
| Afghanistan|2

In [5]:
#Chequeo visual
worldhapprep21.show(10)

+------------+--------------------+------------+------------------------------+------------+------------+---------------------+--------------+-----------------------+----------------------------+----------+-------------------------+------------------------+--------------------------------+----------------------------+-------------------------------------+------------------------------------------+------------------------+---------------------------------------+-------------------+
|Country name|  Regional indicator|Ladder score|Standard error of ladder score|upperwhisker|lowerwhisker|Logged GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|Ladder score in Dystopia|Explained by: Log GDP per capita|Explained by: Social support|Explained by: Healthy life expectancy|Explained by: Freedom to make life choices|Explained by: Generosity|Explained by: Perceptions of corruption|Dystopia + residual|
+------------+--------------

In [6]:
#Chequeo que no haya países con nombre a null
worldhapprep.filter(worldhapprep["Country Name"].isNull()).show()

+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
|Country name|year|Life Ladder|Log GDP per capita|Social support|Healthy life expectancy at birth|Freedom to make life choices|Generosity|Perceptions of corruption|Positive affect|Negative affect|
+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+



In [7]:
#Chequeo que no haya países con nombre a null
worldhapprep21.filter(worldhapprep21['Country name'].isNull()).show()

+------------+------------------+------------+------------------------------+------------+------------+---------------------+--------------+-----------------------+----------------------------+----------+-------------------------+------------------------+--------------------------------+----------------------------+-------------------------------------+------------------------------------------+------------------------+---------------------------------------+-------------------+
|Country name|Regional indicator|Ladder score|Standard error of ladder score|upperwhisker|lowerwhisker|Logged GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|Ladder score in Dystopia|Explained by: Log GDP per capita|Explained by: Social support|Explained by: Healthy life expectancy|Explained by: Freedom to make life choices|Explained by: Generosity|Explained by: Perceptions of corruption|Dystopia + residual|
+------------+------------------

In [8]:
#Imprimo el esquema de los datos para chequearlo:
worldhapprep.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- Life Ladder: double (nullable = true)
 |-- Log GDP per capita: double (nullable = true)
 |-- Social support: double (nullable = true)
 |-- Healthy life expectancy at birth: double (nullable = true)
 |-- Freedom to make life choices: double (nullable = true)
 |-- Generosity: double (nullable = true)
 |-- Perceptions of corruption: double (nullable = true)
 |-- Positive affect: double (nullable = true)
 |-- Negative affect: double (nullable = true)



In [9]:
#Imprimo el esquema de los datos para chequearlo:
worldhapprep21.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- Regional indicator: string (nullable = true)
 |-- Ladder score: double (nullable = true)
 |-- Standard error of ladder score: double (nullable = true)
 |-- upperwhisker: double (nullable = true)
 |-- lowerwhisker: double (nullable = true)
 |-- Logged GDP per capita: double (nullable = true)
 |-- Social support: double (nullable = true)
 |-- Healthy life expectancy: double (nullable = true)
 |-- Freedom to make life choices: double (nullable = true)
 |-- Generosity: double (nullable = true)
 |-- Perceptions of corruption: double (nullable = true)
 |-- Ladder score in Dystopia: double (nullable = true)
 |-- Explained by: Log GDP per capita: double (nullable = true)
 |-- Explained by: Social support: double (nullable = true)
 |-- Explained by: Healthy life expectancy: double (nullable = true)
 |-- Explained by: Freedom to make life choices: double (nullable = true)
 |-- Explained by: Generosity: double (nullable = true)
 |-- Explained 

In [10]:
## 1. ¿Cuál es el país más “feliz” del 2021 según la data? 

preg_1 = worldhapprep21.select("Country name","Ladder score").sort("Ladder score", ascending = False)

preg_1.show(1)

+------------+------------+
|Country name|Ladder score|
+------------+------------+
|     Finland|       7.842|
+------------+------------+
only showing top 1 row



In [11]:
## 2. ¿Cuál es el país más “feliz” del 2021 por continente según la data?

#Considero la columna "Regional indicator" para hacer este análisis
preg_2 = (worldhapprep21.groupBy("Regional indicator")
         .agg(
            max("Ladder score").alias("Max Ladder score"),
             first("Country name").alias("Country")
            )
    .sort(desc("Max Ladder score"))
          )
preg_2.show()

+--------------------+----------------+--------------------+
|  Regional indicator|Max Ladder score|             Country|
+--------------------+----------------+--------------------+
|      Western Europe|           7.842|             Finland|
|North America and...|           7.277|         New Zealand|
|Middle East and N...|           7.157|              Israel|
|Latin America and...|           7.069|          Costa Rica|
|Central and Easte...|           6.965|      Czech Republic|
|           East Asia|           6.584|Taiwan Province o...|
|      Southeast Asia|           6.377|           Singapore|
|Commonwealth of I...|           6.179|          Uzbekistan|
|  Sub-Saharan Africa|           6.049|           Mauritius|
|          South Asia|           5.269|               Nepal|
+--------------------+----------------+--------------------+



In [12]:
## 3. ¿Cuál es el país que más veces ocupó el primer lugar?

#Para usar funciones de ranking, primero hay que hacer window partitioning
windowPart = Window.partitionBy("year").orderBy(desc("Life Ladder"))

rankeddf = worldhapprep.withColumn("Ranking", rank().over(windowPart)).filter(col('Ranking')==1)

preg_3 = rankeddf.groupBy("Country name").agg(
                                      count(col('Ranking')).alias("Total #1 Rnk") 
                                  ).sort(desc("Total #1 Rnk"))

preg_3.show(1)

+------------+------------+
|Country name|Total #1 Rnk|
+------------+------------+
|     Denmark|           7|
+------------+------------+
only showing top 1 row



In [13]:
## 4. ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?

#Aquí también usamos una función de ranking para rankear el índice de felicidad:
windowSpec = Window.partitionBy("year").orderBy(desc("Life ladder"))

happydf = worldhapprep.withColumn("Happiness Rnk Pos", rank().over(windowSpec)) 

preg_4 = happydf.select("Country name", "Log GDP per capita", "Happiness Rnk Pos").filter(happydf.year == 2020)\
    .sort(happydf["Log GDP per capita"].desc(), happydf["Happiness Rnk Pos"])

preg_4.show(1)


+------------+------------------+-----------------+
|Country name|Log GDP per capita|Happiness Rnk Pos|
+------------+------------------+-----------------+
|     Ireland|            11.323|               13|
+------------+------------------+-----------------+
only showing top 1 row



In [14]:
## 6.a ¿Cuál es el país con mayor expectativa de vida (“Healthy life expectancy at birth”)? 

#Nota: hay países con Healthy life expectancy at birth = null, voy a rellenar esos nulos con la media
preg_6a = worldhapprep.na.fill(worldhapprep.select(mean(worldhapprep['Healthy life expectancy at birth'])).collect()[0][0])

preg_6a = worldhapprep.groupBy("Country name") \
             .agg(
                    avg("Healthy life expectancy at birth").alias("Average Life Expec"),
                ).orderBy(desc("Average Life Expec"))
preg_6a.show(1)
            

+------------+------------------+
|Country name|Average Life Expec|
+------------+------------------+
|   Singapore| 75.35846153846154|
+------------+------------------+
only showing top 1 row



In [15]:
## 6.b Y ¿Cuánto tenía en ese indicador en el 2019?

preg_6b = worldhapprep.groupBy("Country name", "year") \
            .agg(
                    avg("Healthy life expectancy at birth").alias("Average Life Exp")
                 ).filter( (col('Country name') == lit("Singapore")) & (col('year') == 2019 ))

preg_6b.show()


+------------+----+----------------+
|Country name|year|Average Life Exp|
+------------+----+----------------+
|   Singapore|2019|            77.1|
+------------+----+----------------+

